# Part 2: Building AI-Powered Semantic Product Search with pgvector and Amazon Bedrock
### Building and Validating Semantic Search

In this second part, we'll implement and test our semantic search functionality. Make sure you've completed Part 1 first.

## Contents
1. Basic Semantic Search
2. Advanced Search with Filters
3. Example Queries and Testing

In [ ]:
# Import required libraries and set up connections
import boto3
import json
import psycopg
from pgvector.psycopg import register_vector
from IPython.display import display, HTML

# Get credentials
client = boto3.client('secretsmanager')
response = client.get_secret_value(SecretId='apg-pgvector-secret-RIV')
database_secrets = json.loads(response['SecretString'])

dbhost = database_secrets['host']
dbport = database_secrets['port']
dbuser = database_secrets['username']
dbpass = database_secrets['password']

bedrock_runtime = boto3.client('bedrock-runtime')

## 1. Basic Semantic Search Implementation

In [ ]:
def generate_embedding(text):
    """Generate embedding for a single text using Amazon Titan"""
    try:
        payload = json.dumps({'inputText': text})
        response = bedrock_runtime.invoke_model(
            body=payload,
            modelId='amazon.titan-embed-text-v2:0',
            accept="application/json",
            contentType="application/json"
        )
        response_body = json.loads(response.get("body").read())
        return response_body.get("embedding")
    except Exception as e:
        print(f"Error generating embedding: {str(e)}")
        return None

def search_products(query, num_results=3):
    """Basic semantic search for products"""
    query_embedding = generate_embedding(query)

    conn = psycopg.connect(
        host=dbhost,
        port=dbport,
        user=dbuser,
        password=dbpass,
        autocommit=True
    )

    # Register vector type
    register_vector(conn)

    # Execute query with type casting
    results = conn.execute("""
        SELECT 
            \"productId\",
            product_description,
            imgUrl,
            stars,
            reviews,
            price,
            category_name,
            1 - (embedding <=> %s::vector) as similarity
        FROM bedrock_integration.product_catalog
        ORDER BY embedding <=> %s::vector
        LIMIT %s;
    """, (query_embedding, query_embedding, num_results)).fetchall()

    display_results(results)
    conn.close()

## 2. Advanced Search with Filters

In [ ]:
def advanced_search(query, category=None, max_price=None, min_stars=None, min_reviews=None, num_results=3):
    """Advanced search with multiple filters"""
    query_embedding = generate_embedding(query)

    conn = psycopg.connect(
        host=dbhost,
        port=dbport,
        user=dbuser,
        password=dbpass,
        autocommit=True
    )

    # Register vector type
    register_vector(conn)

    # Execute query with type casting
    sql = """
        SELECT 
            \"productId\",
            product_description,
            imgUrl,
            stars,
            reviews,
            price,
            category_name,
            1 - (embedding <=> %s::vector) as similarity
        FROM bedrock_integration.product_catalog
        WHERE 1=1
    """
    params = [query_embedding]

    if category:
        sql += " AND category_name = %s"
        params.append(category)
    if max_price:
        sql += " AND price <= %s"
        params.append(max_price)
    if min_stars:
        sql += " AND stars >= %s"
        params.append(min_stars)
    if min_reviews:
        sql += " AND reviews >= %s"
        params.append(min_reviews)

    sql += """
        ORDER BY embedding <=> %s::vector
        LIMIT %s;
    """
    params.extend([query_embedding, num_results])

    results = conn.execute(sql, params).fetchall()
    display_results(results)
    conn.close()

## Helper Function for Result Display

In [ ]:
def display_results(results):
    """Display search results with consistent formatting"""
    html_output = """
    <style>
        .product-card { margin: 15px 0; padding: 20px; border: 1px solid #ddd; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }
        .product-grid { display: grid; grid-template-columns: 200px 1fr; gap: 20px; }
        .product-price { color: #B12704; font-weight: bold; font-size: 1.2em; }
        .product-stars { color: #FFA41C; }
        .product-reviews { color: #007185; }
        .product-category { color: #565959; font-size: 0.9em; }
        .similarity-score { color: #007600; font-weight: bold; }
    </style>
    """

    for row in results:
        similarity = round((row[-1] or 0) * 100, 2)  # Last column is always similarity/score, use 0 if similarity is None, 
        stars = "⭐" * int(row[3]) if row[3] else ""

        html_output += f"""
        <div class="product-card">
            <div class="product-grid">
                <div>
                    <img src="{row[2]}" style="max-width: 180px; height: auto;">
                </div>
                <div>
                    <h3>{row[1][:100]}...</h3>
                    <div class="product-price">${row[5]:.2f}</div>
                    <div class="product-stars">{stars}</div>
                    <div class="product-reviews">({row[4]} reviews)</div>
                    <div class="product-category">Category: {row[6]}</div>
                    <div class="similarity-score">Match Score: {similarity}%</div>
                </div>
            </div>
        </div>
        """

    display(HTML(html_output))

## Example Searches

Let's test our search implementations:

In [ ]:
print("📱 Basic Search: Mobile Accessories")
search_products("I need a phone charger and case")

# Example 1: Smart Home Gadgets
print("🏠 Smart Home Search")
search_products(
    query="smart home automation device for controlling lights and plugs",
    num_results=4
)

# Example 2: Pet Supplies
print("\n🐾 Pet Products Search")
search_products(
    query="dog cat pet supplies food bowl toys accessories",
    num_results=3
)

# Example 3: Tech Accessories
print("\n📱 Tech Gadgets")
search_products(
    query="phone laptop computer accessories charger case protection",
    num_results=4
)

# Example 4: Gaming Equipment
print("\n🎮 Gaming Gear")
search_products(
    query="video game console gaming accessories controllers headset",
    num_results=4
)

# Example 5: Home Organization
print("\n🏡 Home Organization")
search_products(
    query="home organization storage solutions kitchen bathroom bedroom",
    num_results=3
)

# Example 6: Outdoor Activities
print("\n🏕️ Outdoor Equipment")
search_products(
    query="outdoor camping hiking sports recreation gear equipment",
    num_results=4
)

# Example 7: Health Products
print("\n💪 Health and Wellness")
search_products(
    query="multivitamins vitamin D3 omega-3 probiotics protein powder",
    num_results=3
)

# Example 8: Home Security
print("\n🔒 Home Security")
search_products(
    query="home security camera system smart lock monitoring",
    num_results=3
)

## Example searches: Advanced search with filters

In [ ]:
# Example 1: Smart Home Security Products
print("🏠 High-Rated Smart Security Products")
advanced_search(
    query="home security camera system",
    category="Smart Home: Security Cameras and Systems",
    max_price=75,
    min_stars=4.5,
    min_reviews=10,
    num_results=5
)

# Example 2: Cross-Category Smart Home Search
print("\n🏠 Smart Home Starter Kit")
advanced_search(
    query="smart home automation starter",
    category="Smart Home: Voice Assistants and Hubs",
    max_price=50,  # Reasonable price point for starter products
    min_stars=4.4,
    min_reviews=15,
    num_results=4
)

# Example 3: Household Supplies (162 products)
print("\n🏠 Household Supplies")
advanced_search(
    query="household cleaning supplies",
    category="Household Supplies",
    max_price=25,  # Most under $17
    min_stars=3.0,
    num_results=4
)

# Example 4: Kitchen Items (162 products)
print("\n🍳 Kitchen Supplies")
advanced_search(
    query="kitchen cooking supplies",
    category="Kitchen & Dining",
    max_price=30,  # Most under $19
    min_stars=3.0,
    num_results=3
)

# Example 5: Outdoor Recreation (150 products)
print("\n🏕️ Outdoor Gear")
advanced_search(
    query="outdoor recreation camping gear",
    category="Outdoor Recreation",
    max_price=40,  # Most under $23.50
    min_stars=3.5,
    num_results=4
)

# Example 6: Hair Care (102 products)
print("\n💇 Hair Care Products")
advanced_search(
    query="hair care shampoo products",
    category="Hair Care Products",
    max_price=25,  # Most under $15
    min_stars=3.0,
    num_results=3
)

# Example 7: Gift Cards (108 products)
print("\n🎁 Gift Cards")
advanced_search(
    query="gift card certificate",
    category="Gift Cards",
    max_price=50,  # Most under $25
    min_stars=3.0,
    num_results=3
)

# Example 8: Skin Care (97 products)
print("\n✨ Skin Care")
advanced_search(
    query="skin care beauty products",
    category="Skin Care Products",
    max_price=30,  # Most under $19
    min_stars=3.0,
    num_results=3
)

In [ ]:
# Additional test queries
test_queries = [
    "outdoor camping gear",
    "home office essentials"
]

for query in test_queries:
    print(f"\n🔎 Searching for: {query}")
    search_products(query)

## Conclusion

In this workshop, we've implemented two different search approaches:

1. **Basic Semantic Search**: Pure vector similarity search using pgvector's vector distance operator
2. **Advanced Search**: Combining vector search with traditional SQL filters

### Key Takeaways:
- Semantic search provides more natural and contextual results compared to keyword search
- Filtering helps narrow down results to specific requirements while maintaining semantic relevance
- The HNSW index helps optimize vector similarity searches
- The combination of Amazon Bedrock for embeddings and pgvector for storage provides a powerful semantic search solution

### Next Steps:
- Add support for multi-language product descriptions
- Implement faceted search with dynamic aggregations
- Add relevance scoring adjustments based on business rules
- Implement caching for frequently searched queries

### Congratulations! Return to the lab guide and begin the **Blaize Bazaar** lab.